In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

from six.moves import urllib

import pandas as pd
import tensorflow as tf

In [2]:
# Read in data
import pandas as pd
import scipy.io
variable = pd.read_csv('/Users/posnerlab/Desktop/BrainCNN/neonatalinfantbrainstudy/Variables.txt', sep="\t")

infant = pd.read_csv('/Users/posnerlab/Desktop/BrainCNN/neonatalinfantbrainstudy/infant.csv', header=None)
infant_white = pd.read_csv('/Users/posnerlab/Desktop/BrainCNN/neonatalinfantbrainstudy/infant_white.csv', header=None)

data = pd.DataFrame(pd.concat([infant, variable['SSRI']], axis=1))
data_white = pd.concat([infant_white, variable['SSRI']], axis=1)

# Rename edge names
new_cols = dict(zip(data.columns[0:8100], ['edge_' + str(x+1) for x in data.columns[0:8100]]))
data.rename(columns= new_cols, inplace=True)
data_white.rename(columns= new_cols, inplace=True)
data.to_csv('data.csv')
data_white.to_csv('data_white.csv')

In [3]:
# Split train and test data
from sklearn.model_selection import train_test_split
train_infant, test_infant = train_test_split(data, test_size = 0.2)
train_infant_white, test_infant_white = train_test_split(data_white, test_size = 0.2)
train_infant.to_csv('train_infant.csv')
test_infant.to_csv('test_infant.csv')
train_infant_white.to_csv('train_infant_white.csv')
test_infant_white.to_csv('test_infant_white.csv')

In [4]:
# Continuous base columns.
edge = []
for i in range(8099):
    edge.append(tf.contrib.layers.real_valued_column(data.columns[i]))

In [5]:
# Deep columns
deep_columns = []
for i in range(8099):
    deep_columns.append(edge[i])
# Wide columns
wide_columns = []

In [6]:
# Combine deep and wide columns
import tempfile
model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.DNNLinearCombinedClassifier(
    model_dir=model_dir,
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[100, 50])

Instructions for updating:
Please set fix_global_step_increment_bug=True and update training steps in your pipeline. See pydoc for details.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1206667d0>, '_model_dir': '/var/folders/z1/n4w0rk993zl8406dtsghskvr0000gn/T/tmp5vTuM2', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_master': ''}


In [7]:
import pandas as pd
import urllib
COLUMNS = list(data)
LABEL_COLUMN = 'label'
CONTINUOUS_COLUMNS = list(data[0:8099])
#CATEGORICAL_COLUMNS = []

train_file = tempfile.NamedTemporaryFile()
test_file = tempfile.NamedTemporaryFile()
urllib.urlretrieve("/Users/posnerlab/Desktop/BingHan/train_infant.csv", train_file.name)
urllib.urlretrieve("/Users/posnerlab/Desktop/BingHan/test_infant.csv", test_file.name)

('/var/folders/z1/n4w0rk993zl8406dtsghskvr0000gn/T/tmpCxECJK',
 <mimetools.Message instance at 0x1206dbfc8>)

In [8]:
df_train = pd.read_csv(train_file, names=COLUMNS)
df_test = pd.read_csv(test_file, names=COLUMNS)
df_train[LABEL_COLUMN] = (df_train['SSRI'].apply(lambda x: '1' in x)).astype(int)
df_test[LABEL_COLUMN] = (df_test['SSRI'].apply(lambda x: '1' in x)).astype(int)

In [9]:
def input_fn(df):
    """Input builder function."""
    # Creates a dictionary mapping from each continuous feature column name (k) to
    # the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values, shape=[df[k].size, 1]) for k in CONTINUOUS_COLUMNS}
    # Creates a dictionary mapping from each categorical feature column name (k)
    # to the values of that column stored in a tf.SparseTensor.
    #categorical_cols = {
    #    k: tf.SparseTensor(
    #        indices=[[i, 0] for i in range(df[k].size)],
    #        values=df[k].values,
    #        dense_shape=[df[k].size, 1])
    #    for k in CATEGORICAL_COLUMNS}
    # Merges the two dictionaries into one.
    feature_cols = dict(continuous_cols)
    #feature_cols.update(categorical_cols)
    # Converts the label column into a constant Tensor.
    label = tf.constant(df[LABEL_COLUMN].values)
    # Returns the feature columns and the label.
    return feature_cols, label

In [10]:
def train_input_fn():
    return input_fn(df_train)

def eval_input_fn():
    return input_fn(df_test)

In [11]:
m.fit(input_fn=train_input_fn, steps=200)
results = m.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
    print("%s: %s" % (key, results[key]))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.


UnimplementedError: Cast string to float is not supported
	 [[Node: dnn/input_from_feature_columns/input_from_feature_columns/edge_1/ToFloat = Cast[DstT=DT_FLOAT, SrcT=DT_STRING, _device="/job:localhost/replica:0/task:0/cpu:0"](Const)]]

Caused by op u'dnn/input_from_feature_columns/input_from_feature_columns/edge_1/ToFloat', defined at:
  File "/Users/posnerlab/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/posnerlab/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/posnerlab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/posnerlab/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/posnerlab/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start